In [8]:
import osmnx as ox
import networkx as nx
import geopandas as gpd
from shapely.geometry import Point
    # Beispielkoordinaten: Startpunkt und drei Zielpunkte
start_coords = (47.440428621884536, 7.81519133937825)  # Berlin Zentrum
ziel_coords = [
    (47.448323119798, 7.8364773505699095),
    (47.435319786025325, 7.921621395336546),
    (47.38187952630315, 7.765752861771816)
]

# 1. Netzwerk laden (je nach Größe der Region, hier: Umkreis von Startpunkt)
G = ox.load_graphml("../data/ch-bl_walk.graphml")

# 2. Nächstgelegene Knoten im Netzwerk finden
orig_node = ox.distance.nearest_nodes(G, X=start_coords[1], Y=start_coords[0])
ziel_nodes = [ox.distance.nearest_nodes(G, X=lon, Y=lat) for lat, lon in ziel_coords]

# 3. Kürzeste Routen und ihre Dauer berechnen
# Wir gehen davon aus, dass das Netzwerk mit Reisezeit gewichtet ist (z. B. für Fußgänger ca. 5 km/h)
# Optional: Reisedauer als Gewicht setzen
speed_kph = 5
meters_per_minute = speed_kph * 1000 / 60
for u, v, data in G.edges(data=True):
    data['travel_time'] = data['length'] / meters_per_minute

# 4. Zeiten berechnen
zeiten = []
for node in ziel_nodes:
    try:
        route_length = nx.shortest_path_length(G, orig_node, node, weight='travel_time')
        zeiten.append(route_length)  # in Minuten
    except nx.NetworkXNoPath:
        zeiten.append(None)

# 5. Ausgabe als GeoDataFrame
gdf = gpd.GeoDataFrame({
    'zielpunkt': ['Ziel 1', 'Ziel 2', 'Ziel 3'],
    'travel_time_min': zeiten,
    'geometry': [Point(lon, lat) for lat, lon in ziel_coords]
}, crs='EPSG:4326')

print(gdf)

  zielpunkt  travel_time_min                  geometry
0    Ziel 1       137.062842  POINT (7.84678 47.51978)
1    Ziel 2       145.130058  POINT (7.92162 47.43532)
2    Ziel 3       123.763383  POINT (7.76575 47.38188)
